In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, plot_tree

# Initialize parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree

# Precompute constants
dt = T / N
q = (np.exp(r * dt) - d) / (u - d)
disc = np.exp(-r * dt)

# Create binomial tree
stock_tree = np.zeros((N+1, N+1))
option_tree = np.zeros((N+1, N+1))

for i in range(N+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * (u ** j) * (d ** (i - j))

# Initialize option values at maturity
option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)

# Step backwards through the tree to calculate option values
for i in range(N-1, -1, -1):
    for j in range(i+1):
        option_tree[j, i] = disc * (q * option_tree[j, i+1] + (1 - q) * option_tree[j+1, i+1])

# Flatten the stock_tree and option_tree arrays
X = stock_tree.flatten().reshape(-1, 1)
y = option_tree[:, 0]

# Create a decision tree regressor and fit it to the data
tree_reg = DecisionTreeRegressor(max_depth=3)
tree_reg.fit(X, y)

# Plot the decision tree
plt.figure(figsize=(12, 6))
plot_tree(tree_reg, filled=True, feature_names=['Stock Price'], class_names=['Option Value'])
plt.title('Decision Tree for Binomial Option Pricing')
plt.show()


ValueError: Number of labels=4 does not match number of samples=16